In [1]:
import folium
import sqlite3

In [2]:
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

# Create the users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    preferences TEXT
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

# Check if column exists
cursor.execute("PRAGMA table_info(users)")
columns = [column[1] for column in cursor.fetchall()]

# Add column only if it doesn't exist
if 'hometown' not in columns:
    cursor.execute('''
    ALTER TABLE users ADD COLUMN hometown TEXT
    ''')
    print("Added 'hometown' column to users table")
else:
    print("Column 'hometown' already exists")

# Commit the changes and close the connection
conn.commit()
conn.close()

Column 'hometown' already exists


In [4]:
# Example of inserting a user with preferences
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

sample_preferences = "I love skiing and I live in New York City"

# First check if user exists
cursor.execute('SELECT username FROM users WHERE username = ?', ('Sam Controlman',))
if not cursor.fetchone():
    cursor.execute('''
    INSERT INTO users (username, preferences) 
    VALUES (?, ?)
    ''', ('Sam Controlman', sample_preferences))

conn.commit()
conn.close()

In [5]:
# Example of inserting a user with preferences
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

sample_preferences = "I love skiing and I live in New York City"

cursor.execute('''
UPDATE users SET hometown = (?) WHERE username = 'Sam Controlman'
''', ('New York City',))

conn.commit()
conn.close()

In [6]:
# Example of querying the data
conn = sqlite3.connect('user_preferences.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM users')
users = cursor.fetchall()
for user in users:
    print(f"User ID: {user[0]}")
    print(f"Username: {user[1]}")
    print(f"Preferences: {user[2]}")
    print(f"Hometown: {user[3]}\n")

conn.close()

User ID: 1
Username: Sam Controlman
Preferences: Skiing
Hometown: New York City

User ID: 2
Username: jason
Preferences: warm and historic
Hometown: None

User ID: 4
Username: Jason
Preferences: warm places
Hometown: None

User ID: 12
Username: sam altman
Preferences: luxury
Hometown: None

User ID: 16
Username: sam controlman
Preferences: luxury
Hometown: None



In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import sent_tokenize
import sqlite3
import pickle
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
# Initialize the sentence transformer model
encoding_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_chunking(text: str, chunk_size: int = 500, chunk_overlap: int = 50) -> List[str]:
    """Split text into semantic chunks using LangChain's RecursiveCharacterTextSplitter"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    
    return text_splitter.split_text(text)

class CityVectorDB:
    def __init__(self):
        self.dimension = 384  # dimension of all-MiniLM-L6-v2 embeddings
        self.index = faiss.IndexFlatL2(self.dimension)
        self.cities_data = []
        self.chunks_metadata = []
    
    def add_city(self, city: str, description: str):
        # Create chunks
        chunks = semantic_chunking(description)
        
        # Create embeddings for chunks
        embeddings = encoding_model.encode(chunks)
        
        # Add to FAISS index
        self.index.add(np.array(embeddings).astype('float32'))
        
        # Store metadata
        for i, chunk in enumerate(chunks):
            self.chunks_metadata.append({
                'city': city,
                'chunk': chunk,
                'chunk_id': len(self.chunks_metadata) + i
            })
    
    def search(self, query: str, k: int = 3) -> List[Dict]:
        # Create query embedding
        query_vector = encoding_model.encode([query])
        
        # Search in FAISS
        distances, indices = self.index.search(query_vector.astype('float32'), k)
        
        # Get results
        results = []
        for idx in indices[0]:
            if idx != -1:  # FAISS returns -1 if there aren't enough results
                results.append(self.chunks_metadata[idx])
        
        return results
    
    def save(self, filename: str):
        # Save FAISS index
        faiss.write_index(self.index, f"{filename}.index")
        # Save metadata
        with open(f"{filename}.metadata", 'wb') as f:
            pickle.dump(self.chunks_metadata, f)
    
    def load(self, filename: str):
        # Load FAISS index
        self.index = faiss.read_index(f"{filename}.index")
        # Load metadata
        with open(f"{filename}.metadata", 'rb') as f:
            self.chunks_metadata = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Initialize and populate the vector database
cities_data = [
    {
        "city": "New York City, USA",
        "description": """New York City is perfect for urban explorers and culture enthusiasts. 
        It offers world-class museums, Broadway shows, and diverse neighborhoods. 
        The city is known for its shopping, fine dining, and vibrant nightlife. 
        Central Park provides green space for outdoor activities. 
        It's ideal for those who love fast-paced city life and cultural experiences."""
    },
    {
        "city": "Aspen, Colorado, USA",
        "description": """Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers."""
    },
    {
        "city": "Kyoto, Japan",
        "description": """Kyoto is a city steeped in traditional Japanese culture and history.
        It features numerous ancient temples, beautiful gardens, and traditional tea houses.
        The city is famous for its cherry blossoms in spring and vibrant autumn colors.
        Visitors can experience authentic Japanese cuisine and traditional arts.
        It's perfect for those interested in cultural immersion and peaceful environments."""
    },
    {
        "city": "Paris, France",
        "description": """Paris is a city of romance and sophistication. 
        It is renowned for its iconic landmarks such as the Eiffel Tower and Louvre Museum. 
        Visitors can enjoy world-class art, haute cuisine, and chic shopping. 
        The Seine River and charming cafes create a picturesque setting. 
        It's ideal for those seeking romance, art, and gastronomy."""
    },
    {
        "city": "Sydney, Australia",
        "description": """Sydney offers a perfect mix of urban life and natural beauty. 
        It's famous for its iconic Sydney Opera House and Harbour Bridge. 
        Visitors can enjoy beautiful beaches like Bondi and Manly. 
        The city also has vibrant markets, fine dining, and wildlife parks. 
        It's perfect for city explorers and beach lovers."""
    },
    {
        "city": "Cape Town, South Africa",
        "description": """Cape Town is known for its stunning landscapes and cultural diversity. 
        It boasts Table Mountain, beautiful beaches, and vibrant neighborhoods. 
        Visitors can explore vineyards, enjoy wildlife safaris, and experience local cuisine. 
        The city's rich history is evident in sites like Robben Island. 
        It's ideal for adventure seekers and history enthusiasts."""
    },
    {
        "city": "Barcelona, Spain",
        "description": """Barcelona combines unique architecture, rich culture, and Mediterranean charm. 
        It features Gaudi's masterpieces like the Sagrada Familia and Park Güell. 
        Visitors can enjoy bustling markets, beautiful beaches, and vibrant nightlife. 
        The city's Catalan cuisine is a culinary delight. 
        It's perfect for architecture buffs and cultural explorers."""
    },
    {
        "city": "Reykjavik, Iceland",
        "description": """Reykjavik is a gateway to Iceland's natural wonders. 
        It offers unique experiences like the Northern Lights and geothermal lagoons. 
        Visitors can explore volcanic landscapes, waterfalls, and glaciers. 
        The city itself has a vibrant arts scene and welcoming atmosphere. 
        It's ideal for nature lovers and adventure travelers."""
    },
    {
        "city": "Santorini, Greece",
        "description": """Santorini is known for its stunning sunsets and whitewashed buildings. 
        It offers breathtaking views of the Aegean Sea and charming villages. 
        Visitors can explore ancient ruins, volcanic beaches, and enjoy local wines. 
        The island's romantic ambiance makes it a favorite for couples. 
        It's perfect for romantic getaways and scenic relaxation."""
    },
    {
        "city": "Tokyo, Japan",
        "description": """Tokyo is a dynamic city blending tradition with innovation. 
        It offers futuristic skyscrapers, historic temples, and bustling markets. 
        Visitors can enjoy diverse cuisines, vibrant shopping districts, and unique cultural experiences. 
        The city's efficient public transport makes it easy to explore. 
        It's ideal for tech enthusiasts and cultural adventurers."""
    },
    {
        "city": "Rome, Italy",
        "description": """Rome is a treasure trove of history and art. 
        The city is home to ancient landmarks like the Colosseum, Roman Forum, and the Pantheon. 
        Visitors can explore the Vatican City, Sistine Chapel, and St. Peter's Basilica. 
        Rome's vibrant streets are filled with charming cafes and exquisite Italian cuisine. 
        It's perfect for history buffs and lovers of fine food."""
    },
    {
        "city": "Banff, Canada",
        "description": """Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers."""
    },
    {
        "city": "Buenos Aires, Argentina",
        "description": """Buenos Aires is a vibrant city known for its rich culture and lively atmosphere. 
        It features colorful neighborhoods like La Boca, elegant architecture, and historic sites. 
        Visitors can experience tango dancing, world-class steaks, and bustling markets. 
        The city's blend of European and Latin American influences is unique. 
        It's perfect for cultural explorers and food enthusiasts."""
    },
    {
        "city": "Dubai, UAE",
        "description": """Dubai is a futuristic city known for its luxury and innovation. 
        It features iconic landmarks like the Burj Khalifa, Palm Jumeirah, and Dubai Mall. 
        Visitors can enjoy desert safaris, luxury shopping, and world-class dining. 
        The city also offers cultural experiences like traditional souks and dhow cruises. 
        It's ideal for luxury travelers and adventure seekers."""
    },
    {
        "city": "Queenstown, New Zealand",
        "description": """Queenstown is an adventure capital surrounded by stunning natural beauty. 
        It offers activities like bungee jumping, skydiving, and jet boating. 
        Visitors can explore Lake Wakatipu, nearby wineries, and mountain trails. 
        The town's vibrant atmosphere includes cozy cafes and lively bars. 
        It's perfect for thrill-seekers and nature enthusiasts."""
    }
]

In [10]:
# Initialize the vector database
vector_db = CityVectorDB()

# Add cities to the database
for city_data in cities_data:
    vector_db.add_city(city_data["city"], city_data["description"])

# Save the database
vector_db.save("cities_vector_db")

Function to be used in the tool

In [11]:
def get_city_recommendations(username: str, k: int = 3) -> List[Dict]:
    # Connect to SQLite database
    conn = sqlite3.connect('user_preferences.db')
    cursor = conn.cursor()
    
    # Get user preferences
    cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
    result = cursor.fetchone()
    conn.close()
    
    if result:
        user_preferences = result[0]
        # Find matching cities
        matches = vector_db.search(user_preferences, k)
        return matches
    
    return []

In [12]:
# Test the system
test_results = get_city_recommendations('Sam Controlman')
print("\nRecommended Cities based on your preferences:")
for result in test_results:
    print(f"\nCity: {result['city']}")
    print(f"Matching Description: {result['chunk']}")


Recommended Cities based on your preferences:

City: Aspen, Colorado, USA
Matching Description: Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers.

City: Banff, Canada
Matching Description: Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers.

City: Queenstown, New Zealand
Matching Description: Queenstown is an adventure capital surrounded by st

In [13]:
from langchain.tools import BaseTool
from pydantic import Field

class CityRecommendationTool(BaseTool):
    name: str = "city_recommendation_tool"
    description: str = """
    Gets personalized city recommendations based on a user's stored preferences.
    Input should be a username (string) and optionally the number of recommendations (integer).
    Should be called only when we do not know the user's travel destination and not multiple times.
    """
    
    vector_db: CityVectorDB = Field(
        default_factory=CityVectorDB,
        description="Vector database for city matching"
    )
    model: SentenceTransformer = Field(
        default_factory=lambda: SentenceTransformer('all-MiniLM-L6-v2'),
        description="Sentence transformer model for encoding"
    )
    
    def __init__(self, **data):
        super().__init__(**data)
        self.vector_db.load("cities_vector_db")  # Load the saved vector database
    
    def _run(self, username: str, k: int = 3) -> str:
        """Get city recommendations for a user based on their stored preferences"""
        try:
            # Connect to database
            with sqlite3.connect('user_preferences.db') as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
                result = cursor.fetchone()
            
            if not result:
                return f"No preferences found for user: {username}. Please store preferences first."
            
            # Get recommendations using the vector_db's search method
            user_preferences = result[0]
            matches = self.vector_db.search(user_preferences, k)
            
            if not matches:
                return "No matching cities found based on your preferences."
            
            # Format response
            response = [f"Based on your preferences: '{user_preferences}'\n"]
            response.append(f"Here are {len(matches)} recommended cities:\n")
            
            for i, match in enumerate(matches, 1):
                response.append(f"{i}. {match['city']}")
                response.append(f"   Matching aspects: {match['chunk']}\n")
            
            return "\n".join(response)
            
        except Exception as e:
            return f"Error getting recommendations: {str(e)}"
    
    def _arun(self, username: str, k: int = 3) -> str:
        """Async version - Not implemented"""
        raise NotImplementedError("Async implementation not available")

In [14]:
# Initialize the tool
recommendation_tool = CityRecommendationTool()

# Example usage
result = recommendation_tool.run("Sam Controlman")
print(result)

Based on your preferences: 'Skiing'

Here are 3 recommended cities:

1. Aspen, Colorado, USA
   Matching aspects: Aspen is a premier destination for skiing and winter sports enthusiasts. 
        It features world-class ski resorts and stunning mountain views. 
        The town offers luxury shopping and fine dining experiences. 
        In summer, it's perfect for hiking, mountain biking, and outdoor concerts. 
        It's ideal for outdoor adventurers and luxury travelers.

2. Banff, Canada
   Matching aspects: Banff is a haven for outdoor enthusiasts, nestled in the Canadian Rockies. 
        It offers breathtaking mountain views, turquoise lakes like Lake Louise, and abundant wildlife. 
        Visitors can enjoy activities like hiking, skiing, and canoeing. 
        The charming town of Banff features cozy lodges and local shops. 
        It's ideal for nature lovers and adventure seekers.

3. Queenstown, New Zealand
   Matching aspects: Queenstown is an adventure capital surroun

In [15]:
from typing import Optional
import ast


class UserPreferencesTool(BaseTool):
    name: str = "user_preferences_tool"
    description: str = "Manages user preferences for travel recommendations"
    conn: sqlite3.Connection = Field(default_factory=lambda: sqlite3.connect('user_preferences.db', check_same_thread=False), exclude=True)
    cursor: sqlite3.Cursor = Field(default=None, exclude=True)
    
    def __init__(self, **data):
        super().__init__(**data)
        if self.cursor is None:
            self.cursor = self.conn.cursor()
    
    def _get_user_preferences(self, username: str) -> Optional[str]:
        """Check if user exists and return their preferences"""
        self.cursor.execute('SELECT preferences FROM users WHERE username = ?', (username,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    
    def _store_preferences(self, username: str, preferences: str, hometown: str = None) -> None:
        """Store or update user preferences"""
        self.cursor.execute('''
            INSERT INTO users (username, preferences) 
            VALUES (?, ?)
            ON CONFLICT(username) 
            DO UPDATE SET preferences = excluded.preferences
        ''', (username, preferences))
        self.conn.commit()
    
    def _run(self, username: str, preferences: str = None, hometown: str = None) -> str:
        """
        Check if user has preferences, if not prompt for them.
        If preferences are provided, store them.
        Returns the preferences for use with recommendations.
        """
        try:
            # If preferences are provided, store them
            if preferences:
                self._store_preferences(username, preferences, hometown)
                return f"Preferences stored successfully for {username}: {preferences}"
            
            # Otherwise, check for existing preferences
            existing_preferences = self._get_user_preferences(username)
            if existing_preferences:
                return f"Found existing preferences: {existing_preferences}"
            
            # If no preferences found, this message will be sent to the agent
            return "Please tell me about your travel preferences. Consider including:\n" + \
                   "- Activities you enjoy\n" + \
                   "- Climate preferences\n" + \
                   "- Cultural interests\n" + \
                   "- Urban vs. rural settings"
        except Exception as e:
            return f"Error accessing preferences: {str(e)}"

    def update_preferences(self, username: str, new_preferences: str) -> str:
        """Store new user preferences and return confirmation"""
        try:
            self._store_preferences(username, new_preferences)
            return f"Preferences stored successfully for {username}"
        except Exception as e:
            return f"Error storing preferences: {str(e)}"
    
    def __del__(self):
        """Ensure database connection is closed"""
        if hasattr(self, 'conn'):
            self.conn.close()

In [16]:
preferences_tool = UserPreferencesTool()

username = "Sam Controlman"

# Get user preferences
preferences = preferences_tool._get_user_preferences(username)
if preferences:
    print(f"Existing preferences for {username}: {preferences}")
else:
    print(f"No preferences found for {username}.")

Existing preferences for Sam Controlman: Skiing


In [17]:
# !pip install google-search-results

In [18]:
import os
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [19]:
print(os.getenv('SERPAPI_API_KEY'))

680afdfe2534e36965be2913ec4843454ef81b50795335fdbc2d059f99b8d821


In [20]:
serpapi_api_key = '680afdfe2534e36965be2913ec4843454ef81b50795335fdbc2d059f99b8d821'

In [21]:
from serpapi.google_search import GoogleSearch

params = {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "departure_id": "CDG",
    "arrival_id": "AUS",
    "outbound_date": "2024-12-02",
    "return_date": "2024-12-08",
    "currency": "USD",
    "api_key": serpapi_api_key
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'search_metadata': {'id': '674e187fcc4879a0e3da3731',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.json',
  'created_at': '2024-12-02 20:28:47 UTC',
  'processed_at': '2024-12-02 20:28:47 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI0LTEyLTAyagcIARIDQ0RHcgcIARIDQVVTGh4SCjIwMjQtMTItMDhqBwgBEgNBVVNyBwgBEgNDREdCAQFIAXABmAEB',
  'raw_html_file': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.html',
  'prettify_html_file': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.prettify',
  'total_time_taken': 0.47},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'departure_id': 'CDG',
  'arrival_id': 'AUS',
  'outbound_date': '2024-12-02',
  'return_date': '2024-12-08',
  'currency': 'USD'},
 'other_flights': [{'flights': [{'departure_airport': {'name': 'Paris Charles de Gaulle Airpor

In [22]:
params = {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "departure_id": "CDG",
    "arrival_id": "AUS",
    "outbound_date": "2024-12-02",
    "return_date": "2024-12-08",
    "currency": "USD",
    "api_key": serpapi_api_key
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'search_metadata': {'id': '674e187fcc4879a0e3da3731',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.json',
  'created_at': '2024-12-02 20:28:47 UTC',
  'processed_at': '2024-12-02 20:28:47 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI0LTEyLTAyagcIARIDQ0RHcgcIARIDQVVTGh4SCjIwMjQtMTItMDhqBwgBEgNBVVNyBwgBEgNDREdCAQFIAXABmAEB',
  'raw_html_file': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.html',
  'prettify_html_file': 'https://serpapi.com/searches/b09c5931114f4e74/674e187fcc4879a0e3da3731.prettify',
  'total_time_taken': 0.47},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'departure_id': 'CDG',
  'arrival_id': 'AUS',
  'outbound_date': '2024-12-02',
  'return_date': '2024-12-08',
  'currency': 'USD'},
 'other_flights': [{'flights': [{'departure_airport': {'name': 'Paris Charles de Gaulle Airpor

In [23]:
from typing import Optional, Any, Dict, List
from datetime import datetime
from pydantic import Field, BaseModel

# Define the input schema
class FlightSearchInput(BaseModel):
    departure_airport: str = Field(..., description="IATA code of departure airport (e.g., 'JFK')")
    arrival_airport: str = Field(..., description="IATA code of arrival airport (e.g., 'LAX')")
    departure_date: str = Field(..., description="Departure date in YYYY-MM-DD format")
    return_date: str = Field(None, description="Optional return date in YYYY-MM-DD format")

class FlightSearchTool(BaseTool):
    name: str = "flight_search_tool"
    description: str = """
    Searches for flight information between airports. 
    Input should be a dictionary with:
    - departure_airport: IATA code of departure airport (e.g., 'JFK')
    - arrival_airport: IATA code of arrival airport (e.g., 'LAX')
    - departure_date: Date in YYYY-MM-DD format
    """
    args_schema: type[BaseModel] = FlightSearchInput
    api_key: str = Field(..., description="SerpAPI API key")
    
    def _run(self, departure_airport: str, arrival_airport: str, 
             departure_date: str, return_date: str = None) -> str:
        """Execute the flight search"""
        try:
            type_num = 2
            if return_date is not None:
                type_num = 1
            search_params = {
                "engine": "google_flights",
                "hl": "en",
                "gl": "us",
                "departure_id": departure_airport,
                "arrival_id": arrival_airport,
                "outbound_date": departure_date,
                "type": type_num,
                "currency": "USD",
                "api_key": self.api_key
            }
            
            if return_date:
                search_params["return_date"] = return_date
            
            # Rest of your existing search code...
            search = GoogleSearch(search_params)
            results = search.get_dict()
            
            # Format the response
            if 'error' in results:
                return f"Error searching flights: {results['error']}"
            
            # Extract relevant flight information
            best_flights = results.get('best_flights', [])
            if not best_flights:
                return "No flights found for the specified criteria."
            
            # Format response
            response = "Found the following flights:\n"
            for flight in best_flights[:3]:  # Show top 3 results
                price = flight.get('price', 'N/A')
                duration = flight.get('total_duration', 'N/A')
                airline = flight['flights'][0].get('airline', 'N/A')
                departure_time = flight['flights'][0]['departure_airport'].get('time', 'N/A')
                arrival_time = flight['flights'][0]['arrival_airport'].get('time', 'N/A')
                
                response += f"- {airline}: ${price}, Duration: {duration}min ({departure_time} - {arrival_time})\n"
            
            return response
            
        except Exception as e:
            return f"Error processing flight search: {str(e)}"

    def _arun(self, query: str):
        raise NotImplementedError("Async implementation not available")
    
  
    def _validate_date(self, date_str: str) -> bool:
        """Validate date format and ensure it's not in the past"""
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            return date >= datetime.now()
        except ValueError:
            return False

In [24]:
flight_tool = FlightSearchTool(api_key=serpapi_api_key)

# Example input
test_input = {
    'departure_airport': 'JFK',
    'arrival_airport': 'LAX',
    'departure_date': '2024-12-15',
    'return_date': '2024-12-20'
}

result = flight_tool.run(test_input)
print(result)

Found the following flights:
- American: $547, Duration: 382min (2024-12-15 06:00 - 2024-12-15 09:22)
- Delta: $617, Duration: 386min (2024-12-15 08:40 - 2024-12-15 12:06)
- JetBlue: $618, Duration: 380min (2024-12-15 09:00 - 2024-12-15 12:20)



In [ ]:
params = {
  "api_key": serpapi_api_key,
  "engine": "google_hotels",
  "q": "Bali Resorts",
  "hl": "en",
  "gl": "us",
  "check_in_date": "2024-12-03",
  "check_out_date": "2024-12-04",
  "currency": "USD"
}

search = GoogleSearch(params)
results = search.get_dict()

In [ ]:
class HotelSearchInput(BaseModel):
    destination: str = Field(..., description="Name of destination city/area (e.g., 'Bali Resorts')")
    check_in_date: str = Field(..., description="Check-in date in YYYY-MM-DD format")
    check_out_date: str = Field(..., description="Check-out date in YYYY-MM-DD format")
    
class HotelSearchTool(BaseTool):
    name: str = "hotel_search_tool"
    description: str = """
    Searches for hotels at a given destination.
    Input should be a dictionary with:
    - destination: Name of destination city/area (e.g., 'Bali Resorts')
    - check_in_date: Date in YYYY-MM-DD format
    - check_out_date: Date in YYYY-MM-DD format
    """
    args_schema: type[BaseModel] = HotelSearchInput
    api_key: str = Field(..., description="SerpAPI API key")
    
    def _run(self, destination: str, check_in_date: str, check_out_date: str) -> str:
        """Execute the hotel search"""
        try:
            # Validate dates
            if not (self._validate_date(check_in_date) and self._validate_date(check_out_date)):
                return "Error: Please provide valid future dates in YYYY-MM-DD format."
            
            search_params = {
                "api_key": self.api_key,
                "engine": "google_hotels",
                "q": destination,
                "hl": "en",
                "gl": "us",
                "check_in_date": check_in_date,
                "check_out_date": check_out_date,
                "currency": "USD"
            }
            
            search = GoogleSearch(search_params)
            results = search.get_dict()
            
            # Format the response
            if 'error' in results:
                return f"Error searching hotels: {results['error']}"
            
            # Extract hotel information
            hotels = results.get('properties', [])
            if not hotels:
                return f"No hotels found in {destination} for the specified dates."
            
            # Format response
            response = f"Found the following hotels in {destination}:\n\n"
            for hotel in hotels[:5]:  # Show top 5 results
                name = hotel.get('name', 'N/A')
                price = hotel.get('price', {}).get('total', 'N/A')
                rating = hotel.get('rating', 'N/A')
                reviews = hotel.get('reviews', 'N/A')
                description = hotel.get('description', 'No description available')
                
                response += f"🏨 {name}\n"
                response += f"   Price: ${price} total for your stay\n"
                response += f"   Rating: {rating}/5 ({reviews} reviews)\n"
                response += f"   Description: {description[:200]}...\n\n"
            
            return response
            
        except Exception as e:
            return f"Error processing hotel search: {str(e)}"

    def _arun(self, query: str):
        raise NotImplementedError("Async implementation not available")
    
    def _validate_date(self, date_str: str) -> bool:
        """Validate date format and ensure it's not in the past"""
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            return date >= datetime.now()
        except ValueError:
            return False

In [ ]:
hotel_tool = HotelSearchTool(api_key=serpapi_api_key)

# Example usage
test_input = {
    'destination': 'Bali',
    'check_in_date': '2024-12-15',
    'check_out_date': '2024-12-20'
}

result = hotel_tool.run(test_input)
print(result)

In [ ]:
class CalculatorTool(BaseTool):
    name: str = "calculator_tool"
    description: str = """
    Performs basic mathematical calculations. Can handle:
    - Addition (+)
    - Subtraction (-)
    - Multiplication (*)
    - Division (/)
    - Exponents (**)
    - Parentheses ()
    
    Input should be a mathematical expression as a string (e.g., "2 + 2" or "(23 * 4.5) / 2").
    """
    
    def _run(self, expression: str) -> str:
        """Execute the calculation"""
        try:
            # Remove any dangerous functions/attributes
            if any(x in expression.lower() for x in ['import', 'eval', 'exec', 'getattr', '__']):
                return "Error: Invalid expression. Please use only basic mathematical operations."
            
            # Calculate the result
            result = eval(expression, {"__builtins__": {}})
            
            # Format the result
            if isinstance(result, (int, float)):
                # Handle integer results
                if result.is_integer():
                    return f"{int(result)}"
                # Format float results to 4 decimal places
                return f"{result:.4f}".rstrip('0').rstrip('.')
            
            return str(result)
            
        except ZeroDivisionError:
            return "Error: Division by zero"
        except Exception as e:
            return f"Error calculating result: {str(e)}"
    
    def _arun(self, expression: str):
        """Async implementation not available"""
        raise NotImplementedError("Async implementation not available")

In [25]:
from typing import TypedDict, Annotated, Sequence, Union
from langgraph.graph import Graph, StateGraph
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

In [26]:
preferences_tool = UserPreferencesTool()
flight_tool = FlightSearchTool(api_key=serpapi_api_key)
destination_recommendation_tool = CityRecommendationTool()
hotel_tool = HotelSearchTool(api_key=serpapi_api_key)
calculator_tool = CalculatorTool()

tools = [preferences_tool, flight_tool, destination_recommendation_tool, hotel_tool, calculator_tool]

system_prompt = f"""You are a helpful travel assistant named Fred. Follow these steps in order:

1. INITIAL GREETING:
   - Get the user's name
   - Check if they have stored preferences using the user_preferences_tool
   
2. PREFERENCES (only if needed):
   - If no preferences exist, ask about their travel preferences
   - Store their preferences using user_preferences_tool
   
3. CITY RECOMMENDATION:
   - Use city_recommendation_tool EXACTLY ONCE to suggest cities
   - Ask the user if they'd like to search for flights to any of the recommended cities
   
4. FLIGHT SEARCH (when user selects a destination):
   - Ask for specific travel dates
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}
   
5. NEXT STEPS:
   - After showing flight results, ask if they want to:
     a) Search flights for different dates
     b) Search for a different destination
   
6. HOTEL SEARCH (when user selects a destination and a flight):
   - Look for hotels in the destination for the dates of the flight

7. CALCULATOR:
   - If the user asks about the cost of a trip, use the calculator_tool to sum up the selected flight and hotel costs

8. END:
   - Once the user is done with the conversation, thank them and end the conversation
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.
"""

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! I'm Fred, your travel assistant. It's nice to meet you! What's your name?


In [27]:
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
graph = create_react_agent(model, tools=tools, state_modifier=system_prompt)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fred: ================================ Human Message =================================

hi
================================== Ai Message ==================================

👋 Hello! I'm Fred, your travel assistant. What's your name?



You:  sam altman



Fred: ================================ Human Message =================================

sam altman
================================== Ai Message ==================================

Hello Sam Altman! How can I assist you today with your travel plans?



You:  Help me plan


...


Fred: ================================ Human Message =================================

I am based in New York
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_eGIrJwmVJ5kBBxto73cdVRWO)
 Call ID: call_eGIrJwmVJ5kBBxto73cdVRWO
  Args:
    username: Jason
    preferences: warm places
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Jason: warm places
================================== Ai Message ==================================

Great, Jason! I have noted that you are interested in warm places. Where would you like to go for your holiday? Do you have any specific destination in mind?



You:  Dallas maybe?



Fred: ================================ Human Message =================================

Dallas maybe?
================================== Ai Message ==================================
Tool Calls:
  user_preferences_tool (call_Bwc1hekCRTzNvZCrlHwad1n9)
 Call ID: call_Bwc1hekCRTzNvZCrlHwad1n9
  Args:
    username: Jason
    preferences: warm places
    hometown: New York
================================= Tool Message =================================
Name: user_preferences_tool

Preferences stored successfully for Jason: warm places
================================== Ai Message ==================================

Great, Jason! Your preference for warm places has been saved. Now, let's look for flights from New York to Dallas. Let me find the best options for you.
Tool Calls:
  flight_search_tool (call_7R4sHRYDIAPmVzvSGwAHho4x)
 Call ID: call_7R4sHRYDIAPmVzvSGwAHho4x
  Args:
    departure_airport: JFK
    arrival_airport: DFW
    departure_date: 2022-11-15
================================


You:  quit



Fred: Goodbye! Have a great day!


In [28]:
def chat_with_agent(graph):
    print("\nWelcome! I'm Fred, your travel assistant. How can I help you today?")
    
    # Initialize conversation history
    messages = []
    
    while True:
        try:
            # Get user input
            user_input = input("\nYou: ")
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\nFred: Goodbye! Have a great day!")
                break
            
            # Add user message to history and create input state
            messages.append(("user", user_input))
            inputs = {"messages": messages}
            
            # Process through graph and get response
            print("\nFred:", end=" ")
            for s in graph.stream(inputs, stream_mode="values"):
                message = s["messages"][-1]
                if isinstance(message, tuple):
                    # Add to conversation history
                    messages.append(message)
                else:
                    message.pretty_print()
                    
        except KeyboardInterrupt:
            print("\n\nFred: Goodbye! Have a great day!")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try again or type 'exit' to quit.")


# Start the chat
# chat_with_agent(graph)

In [32]:
import gradio as gr
import os

def respond(message, history):
    """Process user message and return agent response"""
    # Convert history to the format expected by the agent
    messages = []
    for human, assistant in history:
        messages.append(("user", human))
        if assistant:  # Skip None responses
            messages.append(("assistant", assistant))
    
    # Add current message
    messages.append(("user", message))
    
    # Process through graph and get response
    inputs = {"messages": messages}
    
    try:
        for s in graph.stream(inputs, stream_mode="values"):
            message = s["messages"][-1]
            if isinstance(message, tuple):
                # Extract just the content from the tuple
                response = message[1]
            else:
                # Extract content from AIMessage or other message types
                response = message.content if hasattr(message, 'content') else str(message)
            yield response
    except Exception as e:
        yield f"Error: {str(e)}\nPlease try again or refresh the page."

# Update the Gradio interface
demo = gr.ChatInterface(
    respond,
    title="Fred the Travel Assistant",
    description="Ask Fred about your next trip!",
    examples=["Hi, I'm planning a trip"],
)

demo.launch()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
